In [9]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 12.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 4.8 MB/s eta 0:00:0000:0100:01


In [30]:
# extract images with pymupdf
import fitz
import os


def extract_images_from_pdf(pdf_path, output_directory):
    doc = fitz.open(pdf_path)
    images = []
    for page in doc:
        for img_index, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_filename = f"image_{page.number+1}_{img_index}.png"
            with open(os.path.join(output_directory, image_filename), "wb") as img_file:
                img_file.write(image_bytes)
            images.append(image_filename)
    return images


output_directory = "../data/processed/images/image_pymupdf"
if not os.path.exists(output_directory):
    os.mkdir(output_directory)
pdf_path = "../data/raw/GPTsareGPTs.pdf"
extracted_images = extract_images_from_pdf(pdf_path, output_directory=output_directory)
print("Extracted Images:", extracted_images)

Extracted Images: ['image_2_0.png', 'image_10_0.png', 'image_10_1.png', 'image_12_0.png', 'image_13_0.png', 'image_13_1.png', 'image_13_2.png', 'image_13_3.png', 'image_15_0.png']


In [27]:
# extract images with pypdf
from pypdf import PdfReader
import os

output_directory = "../data/processed/images/image_pypdf"
if not os.path.exists(output_directory):
    os.mkdir(output_directory)
reader = PdfReader("../data/raw/GPTsareGPTs.pdf")
for page in reader.pages:
    for image in page.images:
        with open(os.path.join(output_directory, image.name), "wb") as fp:
            fp.write(image.data)

Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)


In [1]:
!pip install -U langchain-openai
!pip install --upgrade open-interpreter
!pip install pytesseract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 6.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 7.7 MB/s eta 0:00:00:00:01
  Attempting uninstall: openai
    Found existing installation: openai 0.28.1
    Uninstalling openai-0.28.1:
      Successfully uninstalled openai-0.28.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.40
    Uninstalling langchain-core-0.1.40:
      Successfully uninstalled langchain-core-0.1.40
  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached PyYAML-6.0.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached rich-13.7.1-py3-none-any.whl.metadata (18 kB)
  Using cached wget-3.2-py3-none-any.whl
  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15

In [ ]:
# find relevant images with OCR
import pytesseract
from PIL import Image
import os
from tqdm import tqdm

# Define the global list outside of the function
relevant_images = []


def find_relevant_images_with_ocr(root_dir, keywords):
    global relevant_images  # Use the global keyword to modify the global list
    # Initialize a list to store all the image paths for the progress bar
    image_paths = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(".png"):
                image_paths.append(os.path.join(root, file))

    for image_path in tqdm(image_paths, desc="Processing Images"):
        try:
            text = pytesseract.image_to_string(Image.open(image_path))
            if any(keyword in text.lower() for keyword in keywords):
                relevant_images.append(image_path)
        except Exception as e:
            print(f"Error processing file {image_path}: {e}")


# Example usage
root_directory = "../data/validation/adobe_raw/"
keywords = ["job"]
find_relevant_images_with_ocr(root_directory, keywords)
print(relevant_images)
